# Data Preparation

In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob # for reading the files
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
imgPaths = glob('./Selected/*.jpeg')

In [24]:
allBusinessCard = pd.DataFrame(columns=['id','text'])
for imgPath in tqdm(imgPaths, desc="BusinessCard"):

    _, filename = os.path.split(imgPath)

    # extract data and text
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'), data.split('\n')))
    df = pd.DataFrame(dataList[1:], columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(float)
    df['conf'] = df['conf'].astype(int)

    usefulData = df.query('conf>=30')

    # DataFrame
    businessCard = pd.DataFrame()
    businessCard['text'] = usefulData['text']
    businessCard['id'] = filename
    
    # concatenation (save all the info)
    allBusinessCard = pd.concat((allBusinessCard, businessCard))

BusinessCard: 100%|███████████████████████████| 293/293 [03:46<00:00,  1.29it/s]


In [28]:
allBusinessCard.to_csv('businessCard.csv', index=False)